# Advanced Data Preparation with Python (Car Data)

## Libraries and settings

In [ ]:
# Libraries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

## Importing data

In [ ]:
# Import data from csv to pandas dataframe named 'df'
df = pd.read_csv('./Data/autoscout24_data.csv', sep=";", encoding='utf-8')

# Change the column names to lowercase
df.columns = df.columns.str.lower()

# Display the first 5 rows of the dataframe
df.head()


## Count number of rows and columns in the data frame

In [ ]:
# Count number of rows and columns in the data frame
print("Number of rows: ", df.shape[0])
print("Number of columns: ", df.shape[1])


## Get data types (raw-format from web scraper)

In [ ]:
# Get data types of each column
df.dtypes


## Extract information from raw data using regular expressions (regex)

### Extract numerical values from mileage_raw

In [ ]:
# Extract numerical values (note the 'join' function)
mileage = []
for i in df['mileage_raw']:
    d1 = re.findall("\d+", i)
    try:
        d2 = int(''.join(d1))
    except:
        d2 = None
    mileage.append(d2)

# Save as new variable in the pandas data frame
df['mileage'] = pd.Series(mileage, dtype="Int64")

# Show first records of data frame
df[['mileage_raw', 'mileage']].head()


### Extract numerical values from hp_raw

In [ ]:
# Extract numerical values (note the 'join' function)
hp = []
for i in df['hp_raw']:
    d1 = re.findall("\d+", i)
    try:
        d2 = int(''.join(d1))
    except:
        d2 = None
    hp.append(d2)

# Save as new variable in the pandas data frame
df['hp'] = pd.Series(hp, dtype="Int64")

# Show first records of data frame
df[['hp_raw', 'hp']].head()


### Extract numerical values from price_raw

In [ ]:
# Extract numerical values (note the 'join' function)
price = []
for i in df['price_raw']:
    d1 = re.findall("CHF\s*([\d']+)\.–", i)
    try:
        d2 = d1[0].strip().replace("'", "")
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="Int64")

# Show first records of data frame
df[['price_raw', 'price']].head()



### Get data types of all variables, including the new once

In [ ]:
# Get data types of all variables
df.dtypes


## Count and remove missing values

In [ ]:
# Count and remove missing values
print("Missing values before removing: ", df.isnull().sum().sum())
df = df.dropna()
print("Missing values after removing: ", df.isnull().sum().sum())

# Reset the index of the dataframe
df = df.reset_index(drop=True)


## Count and remove duplicated values

In [ ]:
# Count and remove duplicates
print("Number of duplicates before removing: ", df.duplicated().sum())
df = df.drop_duplicates()
print("Number of duplicates after removing: ", df.duplicated().sum())

# Reset the index of the dataframe
df = df.reset_index(drop=True)


## Use string manipulation methods to create additional variables from the car type

### Change strings in 'type' to uppercase

In [ ]:
# Change strings in 'type' to uppercase and remove leading and trailing whitespaces
df['type'] = df['type'].str.upper().str.strip()

# Display variable 'type'
df['type'].head()


### Calculate length of strings in 'type'

In [ ]:
# Calculate new variable 'type_len' with length of strings in 'type', use .appy()
df['type_len'] = df['type'].apply(len)

# Display variables 'type' and 'type_len'
df[['type', 'type_len']].head()

# Plot histogram of variable 'type_len'
plt.figure(figsize=(7, 4))
sns.histplot(df['type_len'], bins=25, kde=True)
plt.title("Histogram of variable 'type_len'")
plt.xlabel('Type length')
plt.ylabel('Frequency')
plt.show()


### Create binary (0/1) variable 'luxury_car'

In [ ]:
# Create a pattern which can be used to search the variable 'type'
pattern = 'BUGATTI|FERRARI|ROLLS-ROYCE|LAMBORGHINI|BENTLEY|ASTON MARTIN|MASERATI|MCLAREN|PORSCHE'

# Create new variable 'luxury_car' as binary dummy (0/1) variable
df['luxury_car'] = df['type'].str.contains(pat = pattern).astype(int)
print(df['luxury_car'].sum())

# Show values
df[['type','luxury_car',]].loc[df['luxury_car'] == 1]

### Create variable 'price_per_hp'

In [ ]:
# Create variable 'price_per_hp' (round to 2 decimal places)
df['price_per_hp'] = round(df['price'] / df['hp'], 2)

# Display variables 'price', 'hp' and 'price_per_hp'
df[['price', 'hp', 'price_per_hp']].head()


### Include current datetime as time stamp

In [ ]:
# Include current datetime as time stamp in the format 'YYYY-MM-DD HH:MM:SS'
df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df['timestamp'].head()

## Discretization of numerical data

In [ ]:
# Discretization of the variable 'price_per_hp' into 5 bins (note the 'qcut' function)
df['price_per_hp_bin'] = pd.qcut(df['price_per_hp'], 
                                 q=5,  # number of quantiles
                                 labels=['very cheap', 
                                         'cheap', 
                                         'normal', 
                                         'expensive', 
                                         'very expensive'])

# Show histogram of price_per_hp and barchart of variable 'price_per_hp_bin'
plt.figure(figsize=(8, 3))

# Create a 1x2 grid
plt.subplot(1, 2, 1)
sns.histplot(df['price_per_hp'], bins=25, kde=True)
plt.title("Histogram of variable 'price_per_hp'", fontsize=10)
plt.xlabel('Price per HP')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.countplot(data=df, x='price_per_hp_bin', orient='v')
plt.title("Barchart of variable 'price_per_hp_bin'", fontsize=10)
plt.xlabel('Price category')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# Display variables 'price_per_hp' and 'price_per_hp_bin'
df[['price_per_hp', 'price_per_hp_bin']].head()



## One Hot Encoding

In [ ]:
# One Hot Encoding of the variable 'price_per_m2_cat'
try:
    df = pd.get_dummies(df, 
                        columns=['price_per_hp_bin'], 
                        drop_first=False)
except:
    print("Dummy variables already exist")

# Convert boolean values to integers (0 and 1)
df[df.filter(like='price_per_hp_bin').columns] = df.filter(like='price_per_hp_bin').astype(int)

# Show values of the new dummy variables
df.filter(like='price_per_hp_bin').head()


## Scaling

### Min-Max Scaling

In [ ]:
# Min-Max scaling of the variable 'mileage'
df['mileage_scaled_minmax'] = (df['mileage'] - df['mileage'].min()) / (df['mileage'].max() - df['mileage'].min())

# Histogram of the variable 'area' and 'area_scaled'
ax = df[['mileage', 'mileage_scaled_minmax']].hist(bins=25, 
                                      figsize=(7, 3), 
                                      color='steelblue')
                                      
# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Min_Max_Scaled(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

### Max-Absolute Scaling

In [ ]:
# Max-Absolute scaling of the variable 'area'
df['mileage_scaled_max_abs'] = df['mileage'] / df['mileage'].abs().max()

# Histogram of the variable 'area' and 'area_scaled_max_abs'
ax = df[['mileage', 'mileage_scaled_max_abs']].hist(bins=25, 
                                              figsize=(7, 3), 
                                              color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Max_Abs_Scaled(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

### Robust Scaling

In [ ]:
# Robust scaling of the variable 'area'
df['mileage_scaled_robust'] = (df['mileage'] - df['mileage'].median()) / \
                           (df['mileage'].quantile(0.75) \
                           - df['mileage'].quantile(0.25))

# Histogram of the variable 'area' and 'area_scaled_robust'
ax = df[['mileage', 'mileage_scaled_robust']].hist(bins=25, 
                                             figsize=(7, 3), 
                                             color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Scaled_Robust(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

## Standardization

### Z-score Normalization

In [ ]:
# Z-score Normalization of the variable 'area'
df['mileage_scaled_z'] = (df['mileage'] - df['mileage'].mean()) / df['mileage'].std()

# Histogram of the variable 'area' and 'area_scaled_z'
ax = df[['mileage', 'mileage_scaled_z']].hist(bins=25, 
                                        figsize=(7, 3), 
                                        color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Scaled_Z(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

## Transformation

### Log-Transformation

In [ ]:
# Log-Transformation of the variable 'area'
df['mileage_log'] = np.log(df['mileage'])

# Histogram of the variable 'area' and 'area_log'
ax = df[['mileage', 'mileage_log']].hist(bins=25, 
                                   figsize=(7, 3), 
                                   color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Log(Mileage)']):
    axis.set_ylim(0, 1200)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

### SQRT-Transformation

In [ ]:
# SQRT-Transformation of the variable 'area'
df['mileage_sqrt'] = np.sqrt(df['mileage'])

# Histogram of the variable 'area' and 'area_sqrt'
ax = df[['mileage', 'mileage_sqrt']].hist(bins=25, 
                                    figsize=(7, 3), 
                                    color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'SQRT(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

### Box-Cox Transformation

In [ ]:
# Box-Cox Transformation of the variable 'area'
from scipy.stats import boxcox

# Apply Box-Cox transformation
df['mileage_boxcox'], _ = boxcox(df['mileage'])

# Histogram of the variable 'area' and 'area_boxcox'
ax = df[['mileage', 'mileage_boxcox']].hist(bins=25, 
                                      figsize=(7, 3), 
                                      color='steelblue')

# Set the y-axis range for each subplot and add labels
for axis, label in zip(ax.flatten(), ['Mileage', 'Box-Cox(Mileage)']):
    axis.set_ylim(0, 500)
    axis.set_xlabel(label)
    axis.set_ylabel('Frequency')

# Display the plot
plt.show()

## Combining & organizing data

### Reading car data with geocoded addresses

In [ ]:
# Meaning of variables
# offer_id: unique identifier of the offer on autoscout24.ch
# lat: geographical latitude
# lon: geographical longitude
# bfs_number: official municipality id
# bfs_name: official municipality name

# Geocoded data (i.e. data with latitude and longitude)
df_geo = pd.read_csv('./Data/autoscout24_data_geocoded.csv', 
                     sep=';', 
                     encoding='utf-8')

# Show data
df_geo.head()

### Join geo-information to car data using .merge()

In [ ]:
df2 = df.merge(df_geo[['offer_id', 
                       'lat', 
                       'lon', 
                       'bfs_number', 
                       'bfs_name']], 
               on="offer_id",)

# Show data
df2[['offer_id',
     'type',	
     'price',
     'hp',
     'price_per_hp',
     'mileage',	
     'fuel_type',	
     'lat', 
     'lon', 
     'bfs_number', 
     'bfs_name']].head()

### Reading municipality-level data from an .xlsx - file

In [ ]:
# Meaning of variables:
# bfs_number: municipality id
# bfs_name: municipality name
# pop: number of residents
# pop_dens: population density per km2
# frg_pct: percentage foreigners
# emp: number of employees

df_municip = pd.read_excel('./Data/municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

### Join municipality-level data to the car data data using .merge()

In [ ]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp']], 
                on="bfs_number")

# Show data
df3[['offer_id',
     'type',
     'price',
     'mileage',
     'hp',
     'price_per_hp',
     'fuel_type',
     'lat', 
     'lon', 
     'bfs_number', 
     'bfs_name',
     'pop',	
     'pop_dens']].head()

### Sorting data

In [ ]:
# Sorting data by 'price' and 'area' with highest price above (ascending=False)
df3 = df3.sort_values(by=['price', 'mileage'], 
                ascending=False)

df3[['offer_id',
     'type',
     'price',
     'mileage',
     'hp',
     'price_per_hp',
     'fuel_type',
     'lat', 
     'lon', 
     'bfs_number', 
     'bfs_name',
     'pop',	
     'pop_dens']].head()


### Reshaping data

#### Reshaping data using .stack() and .unstack()

In [ ]:
df_sub = df3[['bfs_name', 'mileage', 'price', 'hp']][:5]
print('Original shape')
print(df_sub, '\n')

df_sub_stacked = df_sub.stack()
print('Stacked')
print(df_sub_stacked, '\n')

# Using unstack
print('Unstacked (= back to original shape)')
print(df_sub_stacked.unstack())

#### Reshaping data using .melt()

In [ ]:
df_sub = df3[['mileage', 'price', 'hp']][:5]
print('Original shape')
print(df_sub, '\n')

print('Reshaped using .melt()')
print(pd.melt(df, id_vars=['mileage'], value_vars=['price', 'hp']))

### Pivoting data using .pivot_table()

In [ ]:
pivot_table = pd.pivot_table(df3[['mileage', 'price', 'price_per_hp', 'hp', 'fuel_type']],
                             index=['fuel_type'],
                             values={'price': np.mean, 
                                     'price_per_hp': np.mean, 
                                     'hp': np.mean,
                                     'mileage': np.mean}).round(2)

# Format the values to 2 decimal places with trailing zeros
pivot_table = pivot_table.applymap(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)

# Display the pivot table
pivot_table

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')